<a href="https://colab.research.google.com/github/Haris-09/Python-Programming/blob/main/APIs%2C_Databases%2C_Web_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# APIs Requests

In [ ]:
from urllib.request import Request, urlopen
import json
import requests
import pandas as pd

In [ ]:
api = "https://jsonplaceholder.typicode.com/posts/1"
req = Request(api, headers={'User-Agent': 'Mozilla/5.0'})

with urlopen(req) as response:
  data = response.read()
  string = data.decode()
  print(string)

{
  "userId": 1,
  "id": 1,
  "title": "sunt aut facere repellat provident occaecati excepturi optio reprehenderit",
  "body": "quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit molestiae ut ut quas totam\nnostrum rerum est autem sunt rem eveniet architecto"
}


In [ ]:
url = "https://restcountries.com/v3.1/name/pakistan"
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})

with urlopen(req) as response:
  data = json.loads(response.read().decode())

country = data[0]
print("Country:", country["name"]["common"])
print("Capital:", country["capital"][0])
print("Region:", country["region"])
print("Population:", country["population"])

Country: Pakistan
Capital: Islamabad
Region: Asia
Population: 220892331


In [ ]:
response = requests.get("https://api.open-meteo.com/v1/forecast?latitude=33.6844&longitude=73.0479&current_weather=true")
print(response.url)
print(response.text)

https://api.open-meteo.com/v1/forecast?latitude=33.6844&longitude=73.0479&current_weather=true
{"latitude":33.625,"longitude":73.0,"generationtime_ms":0.05614757537841797,"utc_offset_seconds":0,"timezone":"GMT","timezone_abbreviation":"GMT","elevation":527.0,"current_weather_units":{"time":"iso8601","interval":"seconds","temperature":"°C","windspeed":"km/h","winddirection":"°","is_day":"","weathercode":"wmo code"},"current_weather":{"time":"2025-07-10T14:15","interval":900,"temperature":28.5,"windspeed":7.9,"winddirection":133,"is_day":1,"weathercode":1}}


In [ ]:
# Create dictionary for query parameters
query_params = {
    'latitude': 33.6844,
    'longitude': 73.0479,
    'current_weather': True
}

# Send the GET request using params
response = requests.get('https://api.open-meteo.com/v1/forecast', params=query_params)

# Print the full URL (with query parameters)
print(response.url)
print(response.text)

https://api.open-meteo.com/v1/forecast?latitude=33.6844&longitude=73.0479&current_weather=True
{"latitude":33.625,"longitude":73.0,"generationtime_ms":0.039696693420410156,"utc_offset_seconds":0,"timezone":"GMT","timezone_abbreviation":"GMT","elevation":527.0,"current_weather_units":{"time":"iso8601","interval":"seconds","temperature":"°C","windspeed":"km/h","winddirection":"°","is_day":"","weathercode":"wmo code"},"current_weather":{"time":"2025-07-10T14:15","interval":900,"temperature":28.5,"windspeed":7.9,"winddirection":133,"is_day":1,"weathercode":1}}


In [ ]:
# GET = Retrieve a resource
response = requests.get("https://jsonplaceholder.typicode.com/posts/1") # POST = Create a resource
print(response.text)
print('-----------------------------------------------------------')
response = requests.post("https://jsonplaceholder.typicode.com/posts/1", data={"key": "value"}) # PUT = Update an existing resource
print(response.text)
print('-----------------------------------------------------------')
response = requests.put("https://jsonplaceholder.typicode.com/posts/1", data={"key": "value"}) # DELETE = Remove a resource
print(response.text)
print('-----------------------------------------------------------')
response = requests.delete("https://jsonplaceholder.typicode.com/posts/1")
print(response.text)

{
  "userId": 1,
  "id": 1,
  "title": "sunt aut facere repellat provident occaecati excepturi optio reprehenderit",
  "body": "quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit molestiae ut ut quas totam\nnostrum rerum est autem sunt rem eveniet architecto"
}
-----------------------------------------------------------
{}
-----------------------------------------------------------
{
  "key": "value",
  "id": 1
}
-----------------------------------------------------------
{}


In [ ]:
# Adding headers to a request
response = requests.get(
'https://jsonplaceholder.typicode.com/posts/1', headers={'accept':'application/json'}
)

# Reading response headers
response.headers['content-type']

response.headers.get('content-type')

'application/json; charset=utf-8'

In [ ]:
response.status_code == 200

True

In [ ]:
response = requests.get('https://jsonplaceholder.typicode.com/posts/134542345')
response.status_code == requests.codes.not_found

True

# Connect to your Databases

In [ ]:
from sqlalchemy import create_engine, inspect

# Create a SQLite engine and connect to the database
# engine = create_engine('sqlite:///data/chinook.db')
engine = create_engine('sqlite:///census_nyc.sqlite')
# Establish the connection
connection = engine.connect()
# Create an inspector object
inspector = inspect(engine)
# Get and print the list of table names
table_names = inspector.get_table_names()
print("Tables in the database:", table_names)

Tables in the database: []


In [ ]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, inspect

engine = create_engine('sqlite:///:memory:')

metadata = MetaData()

students = Table(
    'students', metadata,
    Column('id', Integer, primary_key=True),
    Column('name', String),
    Column('age', Integer)
)

metadata.create_all(engine)

inspector = inspect(engine)
print("Tables in DB:", inspector.get_table_names())

students_reflected = Table('students', MetaData(), autoload_with=engine)
print(repr(students_reflected))

Tables in DB: ['students']
Table('students', MetaData(), Column('id', INTEGER(), table=<students>, primary_key=True, nullable=False), Column('name', VARCHAR(), table=<students>), Column('age', INTEGER(), table=<students>), schema=None)


In [ ]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, text

engine = create_engine('sqlite:///:memory:')
metadata = MetaData()
students = Table(
    'students', metadata,
    Column('id', Integer, primary_key=True),
    Column('name', String),
    Column('age', Integer)
)
metadata.create_all(engine)
with engine.connect() as connection:
    connection.execute(students.insert(), [
        {'name': 'Alice', 'age': 22},
        {'name': 'Bob', 'age': 24},
        {'name': 'Charlie', 'age': 23}
    ])

    stmt = text('SELECT * FROM students')
    result_proxy = connection.execute(stmt)
    results = result_proxy.fetchall()

for row in results:
  print(row)

(1, 'Alice', 22)
(2, 'Bob', 24)
(3, 'Charlie', 23)
